In [1]:
import findspark
findspark.init()
import pyspark
import traceback
sc = pyspark.SparkContext(appName="TwitterStreamApp")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [2]:
!export JAVA_HOME='/usr/lib/jvm/java-8-openjdk-amd64'
!export PATH=$JAVA_HOME/bin:$PATH

In [15]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # obtén el contexto spark sql singleton desde el contexto actual
        print("juan")
        sql_context = get_sql_context_instance(rdd.context)
        # convierte el RDD a Row RDD
        print("chu")
        print(rdd.collect())
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # crea un DF desde el Row RDD
        print("zri")
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Registra el marco de data como tabla
        print("for")
        hashtags_df.registerTempTable("hashtags")
        # obtén los 10 mejores hashtags de la tabla utilizando SQL e imprímelos
        print("fai")
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        print("sis")
        hashtag_counts_df.show()
        # llama a este método para preparar los 10 mejores hashtags DF y envíalos
        #print("seve")
        #send_df_to_dashboard(hashtag_counts_df)
    except Exception as ex:
        traceback.print_exception(type(ex), ex, ex.__traceback__)

In [4]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests

In [16]:

# crea un contexto spark con la configuración anterior
sc.setLogLevel("ERROR")
# crea el Contexto Streaming desde el contexto spark visto arriba con intervalo de 2 segundos
ssc = StreamingContext(sc, 2)
# establece un punto de control para permitir la recuperación de RDD
ssc.checkpoint("checkpoint_TwitterApp")
# lee data del puerto 9009
dataStream = ssc.socketTextStream("localhost",8008)
# divide cada Tweet en palabras
words = dataStream.flatMap(lambda line: line.split(" "))
print("AAAAAAAAAAAAAAAAA")
# filtra las palabras para obtener solo hashtags, luego mapea cada hashtag para que sea un par de (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# agrega la cuenta de cada hashtag a su última cuenta
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
print("EEEEEEEEEEEEEEEEEE")
words.saveAsTextFiles('oal')
# procesa cada RDD generado en cada intervalo
tags_totals.foreachRDD(process_rdd)
mySsc = ssc
# comienza la computación de streaming
ssc.start()
# espera que la transmisión termine
ssc.awaitTermination()


AAAAAAAAAAAAAAAAA
EEEEEEEEEEEEEEEEEE
----------- 2021-05-13 22:01:28 -----------
juan
chu
[]
zri


Traceback (most recent call last):
  File "<ipython-input-15-adc59a733fc0>", line 19, in process_rdd
    hashtags_df = sql_context.createDataFrame(row_rdd)
  File "/opt/spark/python/pyspark/sql/context.py", line 321, in createDataFrame
    return self.sparkSession.createDataFrame(data, schema, samplingRatio, verifySchema)
  File "/opt/spark/python/pyspark/sql/session.py", line 605, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema)
  File "/opt/spark/python/pyspark/sql/session.py", line 628, in _create_dataframe
    rdd, schema = self._createFromRDD(data.map(prepare), schema, samplingRatio)
  File "/opt/spark/python/pyspark/sql/session.py", line 425, in _createFromRDD
    struct = self._inferSchema(rdd, samplingRatio, names=schema)
  File "/opt/spark/python/pyspark/sql/session.py", line 396, in _inferSchema
    first = rdd.first()
  File "/opt/spark/python/pyspark/rdd.py", line 1467, in first
    raise ValueError("RDD is empty")
ValueError: 

----------- 2021-05-13 22:01:30 -----------
juan
chu
[]
zri


Traceback (most recent call last):
  File "<ipython-input-15-adc59a733fc0>", line 19, in process_rdd
    hashtags_df = sql_context.createDataFrame(row_rdd)
  File "/opt/spark/python/pyspark/sql/context.py", line 321, in createDataFrame
    return self.sparkSession.createDataFrame(data, schema, samplingRatio, verifySchema)
  File "/opt/spark/python/pyspark/sql/session.py", line 605, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema)
  File "/opt/spark/python/pyspark/sql/session.py", line 628, in _create_dataframe
    rdd, schema = self._createFromRDD(data.map(prepare), schema, samplingRatio)
  File "/opt/spark/python/pyspark/sql/session.py", line 425, in _createFromRDD
    struct = self._inferSchema(rdd, samplingRatio, names=schema)
  File "/opt/spark/python/pyspark/sql/session.py", line 396, in _inferSchema
    first = rdd.first()
  File "/opt/spark/python/pyspark/rdd.py", line 1467, in first
    raise ValueError("RDD is empty")
ValueError: 

----------- 2021-05-13 22:01:32 -----------
juan
chu
[]
zri


...

Traceback (most recent call last):
  File "<ipython-input-15-adc59a733fc0>", line 19, in process_rdd
    hashtags_df = sql_context.createDataFrame(row_rdd)
  File "/opt/spark/python/pyspark/sql/context.py", line 321, in createDataFrame
    return self.sparkSession.createDataFrame(data, schema, samplingRatio, verifySchema)
  File "/opt/spark/python/pyspark/sql/session.py", line 605, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema)
  File "/opt/spark/python/pyspark/sql/session.py", line 628, in _create_dataframe
    rdd, schema = self._createFromRDD(data.map(prepare), schema, samplingRatio)
  File "/opt/spark/python/pyspark/sql/session.py", line 425, in _createFromRDD
    struct = self._inferSchema(rdd, samplingRatio, names=schema)
  File "/opt/spark/python/pyspark/sql/session.py", line 396, in _inferSchema
    first = rdd.first()
  File "/opt/spark/python/pyspark/rdd.py", line 1467, in first
    raise ValueError("RDD is empty")
ValueError: 

----------- 2021-05-13 22:03:32 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:03:34 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:03:36 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:03:38 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:03:40 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|     

----------- 2021-05-13 22:04:52 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:04:54 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:04:56 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:04:58 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+

----------- 2021-05-13 22:05:00 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|     

----------- 2021-05-13 22:06:12 -----------
juan
chu
[('#Twitch', 1)]
zri
for
fai
sis
+-------+-------------+
|hashtag|hashtag_count|
+-------+-------------+
|#Twitch|            1|
+-------+-------------+



KeyboardInterrupt: 

In [17]:
ssc.stop(False)